In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from math import radians, sin, cos, sqrt, atan2

In [ ]:
#  distance function
def haversine(lat1, lon1, lat2, lon2):
    """Calculate Haversine distance (in meters) between two points (lat, lon in degrees)."""
    R = 6371000  # Earth's radius in meters
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * 

In [ ]:
# Load data
df = pd.read_csv('yellow_cab_passenger_mahattan_cleaned.csv')
df1 = pd.read_csv('subway_ridership_manhattan_cleaned.csv'

In [ ]:
# Convert Date and Time columns in df (taxi data)
df['Date'] = pd.to_datetime(df['Date'].astype(str).str.strip(), errors='coerce')
df['Time'] = pd.to_datetime(df['Time'].astype(str).str.strip(), format='%I:%M:%S %p', errors='coerce')
df['Time'] = df['Time'].dt.strftime('%H:%M:%S')  # Format Time as 24-hour string

# Combine Date and Time into a single datetime column
df['datetime'] = pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['datetime'] = df['datetime'].dt.tz_localize(None)  # Ensure timezone-free

# Ensure transit_timestamp in df1 is in datetime format and timezone-free
df1['transit_timestamp'] = pd.to_datetime(df1['transit_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df1['transit_timestamp'] = df1['transit_timestamp'].dt.tz_localize(None)

In [ ]:
# Convert numeric columns and drop NaNs
df['passenger_count'] = pd.to_numeric(df['passenger_count'], errors='coerce')
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df = df.dropna(subset=['passenger_count', 'zone', 'Time', 'Date', 'Latitude', 'Longitude'])
df['passenger_count'] = df['passenger_count'].astype(int)

df1['ridership'] = pd.to_numeric(df1['ridership'], errors='coerce')
df1['latitude'] = pd.to_numeric(df1['latitude'], errors='coerce')
df1['longitude'] = pd.to_numeric(df1['longitude'], errors='coerce')
df1['ridership'] = df1['ridership'].astype(int)

In [ ]:
# Find nearest subway station for each taxi zone using Haversine distance
zone_coords = df[['zone', 'Latitude', 'Longitude']].drop_duplicates(subset=['zone'])
station_coords = df1[['station_complex', 'latitude', 'longitude']].drop_duplicates(subset=['station_complex'])

def find_nearest_station(row, station_coords):
    """Find the nearest station_complex and its distance for a given zone."""
    distances = station_coords.apply(
        lambda x: haversine(row['Latitude'], row['Longitude'], x['latitude'], x['longitude']),
        axis=1
    )
    min_idx = distances.idxmin()
    return pd.Series({
        'nearest_station': station_coords.loc[min_idx, 'station_complex'],
        'distance': distances[min_idx]
    })

# Apply Haversine distance to find nearest station
zone_coords[['nearest_station', 'distance']] = zone_coords.apply(
    lambda row: find_nearest_station(row, station_coords), axis=1
)

In [ ]:
# Merge nearest station into df
df = df.merge(zone_coords[['zone', 'nearest_station']], on='zone', how='left')

In [ ]:
subway_subset = df1[['station_complex', 'transit_timestamp', 'ridership']].rename(
    columns={'station_complex': 'nearest_station', 'transit_timestamp': 'datetime'}
)


In [ ]:
combined_df = df.merge(subway_subset, on=['nearest_station', 'datetime'], how='left')

In [ ]:
#Summarize by zone and datetime to handle duplicates
combined_df = combined_df.groupby(['zone', 'datetime', 'Date', 'Time']).agg({
    'passenger_count': 'sum',
    'ridership': 'sum',
    'nearest_station': 'first'  # Ensure one nearest_station per zone
}).reset_index()


In [ ]:
combined_df['combined_count'] = combined_df['passenger_count'] + combined_df['ridership'].fillna(0)


In [ ]:
# Normalize ridership and passenger_count
scaler = MinMaxScaler()
combined_df[["r_norm", "p_norm"]] = scaler.fit_transform(
    combined_df[["ridership", "passenger_count"]].fillna(0)
)

In [ ]:
# Calculate crowd score
alpha, beta = 0.6, 0.4
min_count = combined_df['combined_count'].min()
max_count = combined_df['combined_count'].max()
if max_count > min_count:
    combined_df["crowd_score"] = (alpha * combined_df["r_norm"] + beta * combined_df["p_norm"]) * 10
else:
    combined_df['crowd_score'] = 0

combined_df['crowd_score'] = combined_df['crowd_score'].round(2)


In [ ]:
result = combined_df[['zone', 'Date', 'Time', 'ridership', 'passenger_count', 'combined_count', 'crowd_score']]
result = result.sort_values(['Date', 'Time', 'zone'])

In [ ]:
result.to_csv('crowd_scores_with_nearest_subway_final.csv', index=False)